# 大模型微调实战

## 安装需要的库

In [1]:
!pip install swanlab

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 20.8 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


## 数据加载

In [2]:
from modelscope.msdatasets import MsDataset
import json
import random

random.seed(42)
print(f"The dataset is loading.")
ds = MsDataset.load('krisfu/delicate_medical_r1_data', subset_name='default', split='train')
data_list = list(ds)
random.shuffle(data_list)

split_idx = int(len(data_list) * 0.9)

train_data = data_list[:split_idx]
val_data = data_list[split_idx:]

with open('train.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

with open('val.jsonl', 'w', encoding='utf-8') as f:
    for item in val_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')
print(f"The dataset has been split successfully.")
print(f"Train Set Size：{len(train_data)}")
print(f"Val Set Size：{len(val_data)}")

The dataset is loading.


2025-08-14 15:23:57,375 - modelscope - INFO - storing https://www.modelscope.cn/api/v1/datasets/krisfu/delicate_medical_r1_data/repo?Source=SDK&Revision=master&FilePath=README.md&View=False in cache at /mnt/workspace/.cache/modelscope/hub/datasets/06b8db8bcdfac6b78df6a26d6969f81867cc1311bc9ce4384b5fb1cffc9504c9
2025-08-14 15:23:57,391 - modelscope - INFO - creating metadata file for /mnt/workspace/.cache/modelscope/hub/datasets/06b8db8bcdfac6b78df6a26d6969f81867cc1311bc9ce4384b5fb1cffc9504c9
2025-08-14 15:24:07,917 - modelscope - INFO - storing https://www.modelscope.cn/api/v1/datasets/krisfu/delicate_medical_r1_data/repo?Source=SDK&Revision=master&FilePath=r1_data_example.jsonl in cache at /mnt/workspace/.cache/modelscope/hub/datasets/downloads/01b7be597488efa04be3c3838a6cb36d08f9b1e99874a112d52790dd4050ad78
2025-08-14 15:24:07,932 - modelscope - INFO - creating metadata file for /mnt/workspace/.cache/modelscope/hub/datasets/downloads/01b7be597488efa04be3c3838a6cb36d08f9b1e99874a112d5

Generating train split: 0 examples [00:00, ? examples/s]

The dataset has been split successfully.
Train Set Size：2166
Val Set Size：241


## 查看数据样本

In [3]:
import json
with open('train.jsonl') as f:
  content = f.readlines()

datas = []
for data in content:
  datas.append(json.loads(data))

datas[0]

{'instruction': '说明测定盐酸甲氧明注射液含量的具体步骤。',
 'question': '医生，我想了解一下如何测定盐酸甲氧明注射液的含量，具体步骤是什么样的？',
 'think': '嗯，用户问的是如何测定盐酸甲氧明注射液的含量，具体步骤是什么。首先，我需要回忆一下相关的分析方法，特别是药典中常用的方法。盐酸甲氧明属于药物分析中的常见成分，通常可能用紫外分光光度法或者HPLC之类的。\n\n首先，用户提到的是紫外-可见分光光度法，这应该是比较直接的方法。那步骤的话，首先应该考虑样品的制备。因为注射液浓度可能较高，需要稀释到合适的浓度范围，使得吸光度在仪器的线性范围内。比如，先取一定量的注射液，然后稀释到某个体积，比如250毫升的量瓶，这样可能是一个步骤。\n\n接下来，可能需要进一步稀释，因为第一次稀释后的溶液可能吸光度还是太高或者太低。比如，从第一次稀释后的溶液中再取一部分，比如10毫升，放到另一个量瓶，比如100毫升，再次稀释。这样两次稀释可以得到合适的浓度。\n\n然后，测定吸光度的时候，波长的选择很重要。盐酸甲氧明的最大吸收波长应该查文献或者药典，比如问题中的答案提到的是290纳米，这可能来自药典的规定。需要确认这个波长是否正确，以及吸收系数是否准确。比如，吸收系数137是否对应C1H1NO3·HCl的计算，可能需要根据分子量和浓度来计算，但用户可能不需要详细计算，只需要步骤。\n\n另外，步骤中的精密量取需要注意，比如使用移液管或者自动进样器，确保量取的准确性。摇匀也是关键步骤，确保溶液均匀，避免局部浓度不均导致误差。\n\n可能还需要考虑空白对照，比如用水代替样品，进行同样的操作，以扣除背景吸光度。不过答案中没有提到，可能在标准方法中已经包含，或者步骤简化了。但作为详细步骤，可能需要补充这一点。\n\n最后，计算含量的时候，根据吸收系数和吸光度，应用朗伯-比尔定律，A=εcl，其中ε是摩尔吸光系数，但这里可能用的是百分吸收系数（1% 1cm），所以需要确认单位是否正确。例如，吸收系数137可能是指在1%浓度（1g/100ml）下，1cm光程的吸光度，这样计算时需要将浓度转换为相应的单位。\n\n可能用户是医生或者药剂师，需要准确的步骤来执行或理解检测流程，所以步骤要清晰，符合药典标准。需要确保步骤中的每个操作都有依据，比如量取的体积

## 加载模型权重

In [4]:
import torch
from modelscope import snapshot_download, AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
# 在modelscope上下载Qwen模型到本地目录下
model_dir = snapshot_download("Qwen/Qwen3-1.7B", cache_dir="./", revision="master")

# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained("./Qwen/Qwen3-1.7B", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./Qwen/Qwen3-1.7B", device_map="auto", torch_dtype=torch.bfloat16)


2025-08-14 15:27:03.855289: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-14 15:27:04.129664: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-14 15:27:05.324863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2025-08-14 15:27:07,980 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-08-14 15:27:08,158 - modelscope - INFO - Got 12 files, start to download ...
Processing 12 items:   0%|          | 0/12 [00:00<?, ?it/s]

Processing 12 items:   8%|▊         | 1/12 [00:01<00:13,  1.19s/it]

Processing 12 items:  25%|██▌       | 3/12 [00:01<00:03,  2.84it/s]

Processing 12 items: 100%|██████████| 12/12 [00:27<00:00,  2.31s/it]
2025-08-14 15:27:35,827 - modelscope - INFO - Download model 'Qwen/Qwen3-1.7B' successfully.
2025-08-14 15:27:35,829 - modelscope - INFO - Creating symbolic link [./Qwen/Qwen3-1.7B].


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
!swanlab login

swanlab: You can find your API key at: https://swanlab.cn/space/~/settings
swanlab: Paste an API key from your profile and hit enter, or press 'CTRL + C' 
to quit: 
Aborted!


In [4]:
import json
import pandas as pd
import torch
import os
from datasets import Dataset
from modelscope import snapshot_download
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
import swanlab
from peft import LoraConfig, TaskType, get_peft_model


# 配置项目参数
os.environ["SWANLAB_PROJECT"] = "qwen3-sft-medical-LORA" 
PROMPT = "你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。"
MAX_LENGTH = 2048

# 配置swanlab实验参数
run = swanlab.init(
    project=os.environ["SWANLAB_PROJECT"],
    config={
    "model": "Qwen/Qwen3-1.7B-LoRA", 
    "prompt": PROMPT,
    "data_max_length": MAX_LENGTH,
    "lora_r": 8, 
    "lora_alpha": 16,
    "lora_dropout": 0.1,
    },
)

def dataset_jsonl_transfer(origin_path, new_path):
    """
    将原始JSONL数据集转换为大模型微调所需的格式
    """
    messages = []
    with open(origin_path, "r", encoding="utf-8") as file:
        for line in file:
            try:
                data = json.loads(line.strip())
                output = f"<|FunctionCallBegin|>{data['think']}<|FunctionCallEnd|> \n {data['answer']}"
                message = {
                    "instruction": PROMPT,
                    "input": data["question"],
                    "output": output
                }
                messages.append(message)
            except Exception as e:
                print(f"处理数据时出错: {e}, 行内容: {line}")
                
    with open(new_path, "w", encoding="utf-8") as file:
        for message in messages:
            file.write(json.dumps(message, ensure_ascii=False) + "\n")

def process_func(example):
    """
    预处理函数：将数据集转换为模型输入格式
    """
    # 构建系统提示和用户输入部分
    instruction = tokenizer(
        f"<|im_start|>system\n{PROMPT}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    
    # 构建模型回应部分
    response = tokenizer(
        f"{example['output']}",
        add_special_tokens=False
    )
    
    # 合并输入和输出，添加结束标记
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    # labels中对输入部分使用-100表示不参与损失计算
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    
    # 截断过长序列
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

def predict(messages, model, tokenizer):
    """
    预测函数：根据输入消息生成模型回应
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # 应用聊天模板格式化输入
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # 转换为模型输入格式
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    # 生成回应
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=MAX_LENGTH,
        temperature=0.7,  # 控制生成多样性
        do_sample=True
    )
    
    # 提取生成的部分（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    # 解码为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

# 主程序执行
if __name__ == "__main__":
    # 下载模型（如果本地没有）
    model_dir = snapshot_download(
        "Qwen/Qwen3-1.7B",
        cache_dir="/mnt/workspace",
        revision="master"
    )
    
    # 加载tokenizer和模型
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir,
        use_fast=False,
        trust_remote_code=True
    )

    # 对于LoRA微调，如果模型本身没有pad_token，显式指定一个
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    

    model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    # 开启梯度检查点所需配置 
    model.enable_input_require_grads()

    # ==================== 配置和应用LoRA ====================
    # 1. 定义LoRA配置
    config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,  # 任务类型：因果语言模型
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # 指定要应用LoRA的模块
        r=8,                           # LoRA的秩，常用8, 16, 32。越小参数越少
        lora_alpha=16,                 # LoRA的缩放因子，通常是r的两倍
        lora_dropout=0.1,              # Dropout比例
    )

    # 2. 使用get_peft_model将基础模型包装为PEFT模型
    model = get_peft_model(model, config)

    # 3. 打印可训练参数的数量和比例，以确认LoRA生效
    model.print_trainable_parameters()
    
    # 数据集路径配置 
    train_dataset_path = "train.jsonl"
    test_dataset_path = "val.jsonl"
    train_jsonl_new_path = "train_format.jsonl"
    test_jsonl_new_path = "val_format.jsonl"
    
    # 转换数据集格式 
    if not os.path.exists(train_jsonl_new_path):
        print(f"转换训练集: {train_dataset_path} -> {train_jsonl_new_path}")
        dataset_jsonl_transfer(train_dataset_path, train_jsonl_new_path)
    
    if not os.path.exists(test_jsonl_new_path):
        print(f"转换验证集: {test_dataset_path} -> {test_jsonl_new_path}")
        dataset_jsonl_transfer(test_dataset_path, test_jsonl_new_path)
    
    # 加载并预处理训练集 
    print("加载训练集并进行预处理...")
    train_df = pd.read_json(train_jsonl_new_path, lines=True)
    train_ds = Dataset.from_pandas(train_df)
    train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)
    
    # 加载并预处理验证集
    print("加载验证集并进行预处理...")
    eval_df = pd.read_json(test_jsonl_new_path, lines=True)
    eval_ds = Dataset.from_pandas(eval_df)
    eval_dataset = eval_ds.map(process_func, remove_columns=eval_ds.column_names)
    
    # 配置训练参数
    training_args = TrainingArguments(
        output_dir="/mnt/workspace/Qwen3-1.7B-LoRA", 
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=2,
        eval_strategy="steps",
        eval_steps=100,
        logging_steps=10,
        num_train_epochs=4,
        save_steps=400,
        learning_rate=5e-5,
        save_on_each_node=True,
        gradient_checkpointing=True,
        report_to="swanlab",
        run_name="qwen3-1.7B-medical-lora",
        bf16=True,
        optim="adamw_torch_fused",
    )

    # 初始化Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
    )

    # 开始训练
    print("开始训练...")
    trainer.train()

    # 保存最终模型
    # 对于PEFT模型，只保存适配器部分
    print("保存训练好的LoRA适配器...")
    final_lora_path = os.path.join(training_args.output_dir, "final_lora_adapter")
    trainer.model.save_pretrained(final_lora_path)
    # 同时也可以保存tokenizer
    tokenizer.save_pretrained(final_lora_path)

    # ==================== 预测前需要合并模型 ====================
    # 在进行预测或部署时，需要将LoRA权重与基础模型合并
    from peft import PeftModel
    # 重新加载基础模型
    base_model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    # 加载LoRA适配器并与基础模型合并
    model = PeftModel.from_pretrained(base_model, final_lora_path)
    model = model.merge_and_unload() # 合并权重
    

    # 在测试集上进行预测并记录结果
    print("在测试集上进行预测...")
    test_df = pd.read_json(test_jsonl_new_path, lines=True)[:3]  # 取前3条测试
    test_text_list = []
    
    for index, row in test_df.iterrows():
        messages = [
            {"role": "system", "content": row['instruction']},
            {"role": "user", "content": row['input']}
        ]
        
        try:
            response = predict(messages, model, tokenizer)
            response_text = f"""
            样本 {index + 1}:
            问题: {row['input']}
            模型回答: {response}
            """
            print(response_text)
            test_text_list.append(swanlab.Text(response_text))
        except Exception as e:
            print(f"预测样本 {index + 1} 时出错: {e}")
    
    # 记录预测结果到swanlab
    swanlab.log({"预测结果": test_text_list})
    swanlab.finish()
    print("训练和评估完成!")

Output()

swanlab: Tracking run with swanlab version 0.6.8

swanlab: Run data will be saved locally in /mnt/workspace/swanlog/run-20250815_091543-n4nb1emz9pcnw4xeomu4c

swanlab: 👋 Hi csr,welcome to swanlab!

swanlab: Syncing run rat-2 to the cloud

...

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


在测试集上进行预测...

            样本 1:
            问题: 1895年德国物理学教授伦琴的发现对医学影像学的发展有何具体影响？请从技术进步、学科建立和临床应用三个方面进行分析。
            模型回答: <|FunctionCallBegin|>嗯，用户问的是1895年伦琴发现X射线对医学影像学的影响，需要从技术进步、学科建立和临床应用三个方面来分析。首先，我得先回忆一下X射线的基本知识，伦琴确实是在1895年发现的，这可能就是用户提到的那一年。然后，我需要分三个部分来展开思考。

首先，技术进步方面，X射线是第一种基于辐射的影像技术，对吧？所以这应该算是技术上的突破。之前可能都是通过光学显微镜或者X光片的其他方法，而X射线的发现让医生可以不用显微镜就能看到内部结构，这应该是技术上的重大进展。另外，可能还有后续的技术发展，比如X射线的进一步应用，比如数字影像之类的，但用户的问题可能只需要1895年的直接影响，所以可能不需要太深入后续发展，但得提到它作为基础。

然后是学科建立方面，伦琴的发现应该促使医学影像学成为独立学科。之前可能更多是解剖学或者外科的分支，而影像学作为独立学科，需要系统的研究和理论发展。比如，可能成立了专门的医学影像学机构，或者培养相关人才，推动理论体系的建立。另外，可能还有相关学科如放射学、内窥镜等的兴起，都是建立在X射线基础上的。

临床应用方面，X射线的直接应用就是诊断疾病，比如骨折、肺结节等。这使得医生能够更早发现疾病，进行及时治疗。比如，肺部疾病可以通过X光拍片诊断，这改变了传统的诊断方法，比如依赖症状和体征，或者需要手术检查。此外，可能还有后续的发展，比如CT、MRI等，但同样，1895年的发现直接推动了这些技术的出现，作为基础技术。

不过，用户可能需要更具体的例子，比如在学科建立方面，伦琴的工作如何引发其他科学家的研究，比如后来的医生和科学家如何将X射线应用到医学中，建立理论，或者形成教学体系。比如，可能在柏林大学设立医学影像学的课程，或者有专门的教材和研究机构。

另外，临床应用部分，除了诊断疾病，可能还有其他应用，比如治疗手段，比如X射线引导下的手术，或者治疗某些疾病，但可能主要还是诊断。不过需要确认这一点是否准确，是否有其他应用。

还要注意时间点，用户明确提到1895年，所以需要

swanlab: 🏠 View project at https://swanlab.cn/@csr/qwen3-sft-medical-LORA

swanlab: 🚀 View run at https://swanlab.cn/@csr/qwen3-sft-medical-LORA/runs/n4nb1emz9pcnw4xeomu4c

训练和评估完成!


In [1]:
from peft import PeftModel
import os
import json
import pandas as pd
import torch
from datasets import Dataset
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
MAX_LENGTH = 2048

def predict_stream_simple(messages, model, tokenizer):
    """
    流式预测函数：使用TextStreamer直接在控制台打印输出
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # 应用聊天模板格式化输入
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # 转换为模型输入格式
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    # 创建 TextStreamer
    # skip_prompt=True 会跳过打印输入的部分，只打印模型生成的内容
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    # 流式生成回应，这里会直接打印到屏幕
    # 注意：generate函数现在会阻塞直到生成结束，但过程中会流式打印
    _ = model.generate(
        **model_inputs,
        max_new_tokens=MAX_LENGTH,
        temperature=0.7,
        do_sample=True,
        streamer=streamer # 关键参数：传入streamer
    )


if __name__ == "__main__":
    # 确保你的路径是正确的
    final_lora_path = 'Qwen3-1.7B-LoRA/final_lora_adapter'
    model_dir = "Qwen/Qwen3-1.7B"

    # 检查路径是否存在
    if not os.path.exists(final_lora_path):
        raise FileNotFoundError(f"LoRA adapter path not found: {final_lora_path}")
    if not os.path.exists(model_dir):
         raise FileNotFoundError(f"Base model path not found: {model_dir}")

    # 重新加载基础模型
    base_model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    # 加载LoRA适配器并与基础模型合并
    model = PeftModel.from_pretrained(base_model, final_lora_path)
    model = model.merge_and_unload() # 合并权重
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir,
        use_fast=False,
        trust_remote_code=True
    )
    
    instruction = "你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。"
    question = "你好，医生，我现在头痛的厉害，吃什么药可以缓解呢"
    messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": question}
    ]
    
    print("回答：", end="", flush=True)
    # 直接调用新的函数
    predict_stream_simple(messages, model, tokenizer)
    print()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

回答：<|FunctionCallBegin|>嗯，用户问的是头痛的常见原因有哪些，以及如何根据不同的原因选择合适的药物。首先，我需要回忆一下常见的头痛类型，比如紧张性头痛、偏头痛、丛集性头痛，还有其他类型比如血管性头痛或者颅内压增高引起的。然后，针对每种头痛类型，常见的治疗方法是什么，药物上有什么区别。

用户还提到自己头痛厉害，想知道吃什么药，所以可能需要先判断头痛的原因，再给出建议。但用户可能不太清楚症状的具体情况，所以需要先询问症状类型，比如是紧张型还是偏头痛，这样药物选择才能更准确。不过用户可能希望得到一般性的指导，所以需要平衡专业性和易懂性。

接下来，我需要列出常见的头痛原因，确保覆盖主要类型。比如紧张性头痛通常与压力有关，可能需要非甾体抗炎药；偏头痛可能需要抗凝血药或者钙通道阻滞剂；丛集性头痛可能需要特定的药物如氟桂利嗪或曲普坦类。还有血管性头痛可能需要血管收缩剂或者钙通道调节剂。颅内压增高可能需要降颅压药物，但这种情况需要专业诊断。

然后，根据不同的原因，药物的分类也要对应。比如，钙通道阻滞剂（如氟桂利嗪）常用于偏头痛和血管性头痛。抗凝血药如华法林用于偏头痛，因为偏头痛可能有血栓形成的风险。曲普坦类如舒马曲普坦用于急性偏头痛发作。血管扩张剂如氟桂利嗪可能用于血管性头痛。降颅压药物如甘露醇用于颅内压增高的情况。

用户可能需要知道在什么情况下使用哪种药物，比如急性发作时用曲普坦，长期预防可能用氟桂利嗪。另外，如果症状严重或持续，应该尽快就医，避免自行用药延误治疗。

还要注意用户可能没有明确说明症状的具体类型，所以回答中需要建议他们详细描述症状，以便医生判断。同时，强调药物的选择需要专业医生的指导，避免自行用药的风险。

总结下来，回答的结构应该是先分类头痛原因，然后分点说明每种原因对应的药物，最后建议就医和描述症状。这样既专业又易懂，符合用户的需求。
<|FunctionCallEnd|> 
 您好，头痛的原因有很多，比如紧张性头痛、偏头痛、丛集性头痛、血管性头痛、颅内压增高等。针对不同的原因，您可以选择相应的药物。例如，偏头痛发作时，可以使用钙通道阻滞剂或抗凝血药；血管性头痛可以使用血管扩张剂或钙通道调节剂；而颅内压增高则需要使用降颅压药物。不过，为了确保安全和有效，建议您详细描述您的症状，以便我们为您选择最适合的药物。如果头痛严重或持续不缓解

In [ ]:
import json
import pandas as pd
import torch
from datasets import Dataset
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
MAX_LENGTH = 2048

def predict_and_stream(messages, model, tokenizer):
    """
    修改后的流式预测函数：
    1. 使用TextStreamer直接在控制台打印输出。
    2. 返回模型生成的完整回复文本，用于添加到对话历史中。
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # 应用聊天模板格式化输入
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # 转换为模型输入格式
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    input_ids = model_inputs.input_ids
    input_length = input_ids.shape[1] # 记录输入内容的长度

    # 创建 TextStreamer
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    
    # 使用 streamer 进行流式输出，同时接收完整的生成结果
    generated_ids = model.generate(
        input_ids,
        max_new_tokens=MAX_LENGTH,
        temperature=0.7,
        do_sample=True,
        streamer=streamer
    )
    
    # 从返回的 generated_ids 中解码出完整的回复
    # 需要去掉输入部分，只保留新生成的部分
    response_ids = generated_ids[0][input_length:]
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True)
    
    return response_text


if __name__ == "__main__":
    # --- 模型加载部分 (与之前相同) ---
    final_lora_path = 'Qwen3-1.7B-LoRA/final_lora_adapter'
    model_dir = "Qwen/Qwen3-1.7B"

    if not os.path.exists(final_lora_path):
        raise FileNotFoundError(f"LoRA adapter path not found: {final_lora_path}")
    if not os.path.exists(model_dir):
         raise FileNotFoundError(f"Base model path not found: {model_dir}")

    base_model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    model = PeftModel.from_pretrained(base_model, final_lora_path)
    model = model.merge_and_unload()
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir,
        use_fast=False,
        trust_remote_code=True
    )
    
    # --- 聊天循环部分 (核心修改) ---
    
    # 1. 初始化对话历史，包含系统指令
    instruction = "你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。"
    messages = [
        {"role": "system", "content": instruction}
    ]
    
    print("医学专家已上线。输入 'exit' 或 'quit' 退出程序。")
    
    # 2. 启动一个无限循环来进行对话
    while True:
        # 获取用户输入
        question = input("你: ")
        
        if question.lower() in ["exit", "quit"]:
            print("感谢您的使用！")
            break
            
        # 3. 将用户的提问添加到历史记录
        messages.append({"role": "user", "content": question})
        
        print("专家: ", end="", flush=True)
        
        # 4. 调用模型，传入完整的对话历史
        # 函数会流式打印答案，并返回完整的答案文本
        assistant_response = predict_and_stream(messages, model, tokenizer)
        
        # 换行，以便下一次输入
        print() 
        
        # 5. 将模型的回答也添加到历史记录，为下一次对话做准备
        messages.append({"role": "assistant", "content": assistant_response})
